Q1. Embedding the query
Embed the query: 'I just discovered the course. Can I join now?'. Use the 'jinaai/jina-embeddings-v2-small-en' model.

You should get a numpy array of size 512.

What's the minimal value in this array?

-0.51
-0.11
0
0.51
Cosine similarity
The vectors that our embedding model returns are already normalized: their length is 1.0.

You can chech that by using the norm function:

import numpy as np
np.linalg.norm(q)
Which means that we can simply compute the dot product between two vectors to learn the cosine similarity between them.

For example, if you compute the cosine of the query vector with itself, the result will be 1.0:

q.dot(q)m

In [ ]:
!pip install fastembed

In [10]:

from fastembed import TextEmbedding
import numpy as np


In [5]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
query = 'I just discovered the course. Can I join now?' 
embedding_dimensiality = 512

In [6]:
embedding_model = TextEmbedding(model_name=model_handle)

In [7]:
#sanity check
embedding_model.model_name

'jinaai/jina-embeddings-v2-small-en'

In [21]:
q_list = list(embedding_model.embed(query))
q = q_list[0] # Get the first query
#print the length of the query vector
print(f'length of the query vector: {len(q)}')
#print the minimum value in the query vector
print(f'the minimum value in the query vector: {min(q)}')

#print the norm of the query vector
print(f'the norm of the query vector: {np.linalg.norm(q)}')
#print the dot product of the query vector with itself
print(f'the dot product of the query vector with itself: {q.dot(q)}')

length of the query vector: 512
the minimum value in the query vector: -0.11726373551188797
the norm of the query vector: 1.0
the dot product of the query vector with itself: 1.0


Q2. Cosine similarity with another vector
Now let's embed this document:

doc = 'Can I still join the course after the start date?'
What's the cosine similarity between the vector for the query and the vector for the document?

0.3
0.5
0.7
0.9

In [13]:
doc = 'Can I still join the course after the start date?'

In [ ]:
doc_list = list(embedding_model.embed(doc))
d = doc_list[0] # Get the first doc
#print the length of the doc vector
print(f'length of the doc vector: {len(d)}')
#print the minimum value in the doc vector
print(f'the minimum value in the doc vector: {min(d)}')

#print the norm of the doc vector
print(f'the norm of the doc vector: {np.linalg.norm(d)}')
#print the vector similarity between the query and the doc
print(f'the dot product of the query vector with the doc vector: {q.dot(d)}')

length of the doc vector: 512
the minimum value in the doc vector: -0.12396319741919225
the norm of the doc vector: 0.9999999999999999
the dot product of the query vector with the doc vector: 0.9008528856818037


Q3. Ranking by cosine

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0):

0
1
2
3
4
Hint: if you put all the embeddings of the text field in one matrix V (a single 2-dimensional numpy array), then computing the cosine becomes a matrix multiplication:



In [73]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [74]:
V_Q3 = list(embedding_model.embed(list(map(lambda doc : doc['text'], documents))))

highest = 0
for i, v_q3 in enumerate(V_Q3):
    if v_q3.dot(q) > highest:
        highest = v_q3.dot(q)
        index = i
    print(f'Index {i}:  dot product of the document vector with query: {v_q3.dot(q)}')

#print highest index and the dot product value
print(f'Highest index: {index}, dot product value: {highest}')


Index 0:  dot product of the document vector with query: 0.7629684493123693
Index 1:  dot product of the document vector with query: 0.8182378361919107
Index 2:  dot product of the document vector with query: 0.8085397290762828
Index 3:  dot product of the document vector with query: 0.7133078539597724
Index 4:  dot product of the document vector with query: 0.7304499528359614
Highest index: 1, dot product value: 0.8182378361919107


Q4. Ranking by cosine, version two


Now let's calculate a new field, which is a concatenation of question and text:

full_text = doc['question'] + ' ' + doc['text']
Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

0
1
2
3
4
Is it different from Q3? If yes, why?

In [75]:
full_text =list(map(lambda doc : doc['question'] + ' ' + doc['text'], documents))


In [76]:
V_Q4 = list(embedding_model.embed(list(map(lambda doc : doc['question'] + ' ' + doc['text'], documents))))

highest = 0
for i, v_q4 in enumerate(V_Q4):
    if v_q4.dot(q) > highest:
        highest = v_q4.dot(q)
        index = i
    print(f'Index {i}:  dot product of the document vector with query: {v_q4.dot(q)}')

#print highest index and the dot product value
print(f'Highest index: {index}, dot product value: {highest}')

Index 0:  dot product of the document vector with query: 0.8514542983476503
Index 1:  dot product of the document vector with query: 0.8436594058703555
Index 2:  dot product of the document vector with query: 0.8408287206920106
Index 3:  dot product of the document vector with query: 0.7755157689596764
Index 4:  dot product of the document vector with query: 0.8086007942697778
Highest index: 0, dot product value: 0.8514542983476503


In [50]:
for doc in documents:
    print(f"Question: {doc['question']}")
    print(f"Text: {doc['text']}")
    print("\n")

Question: Course - Can I still join the course after the start date?
Text: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


Question: Course - Can I follow the course after it finishes?
Text: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.


Question: Course - When will the course start?
Text: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the c

WHY: Because the query has very similar wording to the question in the documents, the model is able to find the most relevant document based on the question text alone.

Q5. Selecting the embedding model

Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

128
256
384
512
One of these models is BAAI/bge-small-en. Let's use it.

In [77]:
smallest_model = min(TextEmbedding.list_supported_models(), key=lambda model: model['dim'])
print (f"Smallest dimensiality model: {smallest_model['model']} with dimension {smallest_model['dim']}")


Smallest dimensiality model: BAAI/bge-small-en with dimension 384


Q6. Indexing with qdrant (2 points)

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp:

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

What's the highest score in the results? (The score for the first returned record):

0.97
0.87
0.77
0.67

In [78]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents_q6 = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents_q6.append(doc)

In [ ]:
!python -m pip install -q "qdrant-client[fastembed]>=1.14.2"

In [61]:
from qdrant_client import QdrantClient, models

In [62]:
qd_client = QdrantClient("http://localhost:6333")

In [81]:
EMBEDDING_DIMENSIONALITY = 384
model_handle_q6 = "BAAI/bge-small-en"
collection_name = "zoomcamp-faq"

In [ ]:
qd_client.delete_collection(collection_name=collection_name)


True

In [67]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [82]:
points = []

for i, doc in enumerate(documents_q6):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model=model_handle_q6)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [83]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████| 5/5 [00:22<00:00,  4.43s/it]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [93]:
def vector_search(question):
    print('vector_search is used')
    
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle_q6 
        ),
        limit=5,
        with_payload=True
    )
    
    results = []
    
    for point in query_points.points:
        results.append(point)
    
    return results

In [97]:
results = vector_search(query)
#print the higshest score and the question from the results
print(f'Highest score: {results[0].score}')

vector_search is used
Highest score: 0.8703172
